In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("Dataset-Olusturmak") \
.config("spark.executor.memory","4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

sc = spark.sparkContext

In [4]:
dosya_yolu="/home/taha/Downloads/OnlineRetail.csv"

In [5]:
retail_RDD=sc.textFile(dosya_yolu)

In [6]:
retail_RDD.take(5)

['InvoiceNo;StockCode;Description;Quantity;InvoiceDate;UnitPrice;CustomerID;Country',
 '536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom']

## Basligi kaldiralim

In [7]:
retail_RDD_without_title=retail_RDD.filter(lambda x:'InvoiceNo' not in x)

In [8]:
retail_RDD_without_title.take(5)

['536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom',
 '536365;71053;WHITE METAL LANTERN;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84406B;CREAM CUPID HEARTS COAT HANGER;8;1.12.2010 08:26;2,75;17850;United Kingdom',
 '536365;84029G;KNITTED UNION FLAG HOT WATER BOTTLE;6;1.12.2010 08:26;3,39;17850;United Kingdom',
 '536365;84029E;RED WOOLLY HOTTIE WHITE HEART.;6;1.12.2010 08:26;3,39;17850;United Kingdom']

## amac satisi iptal olan urunlerin maliyetlerini bulma

In [13]:
def cancelled_price(line):
    is_cancelled = True if(line.split(';')[0].startswith('C')) else False
    quantity = float(line.split(';')[3])
    price = float(line.split(';')[5].replace(',','.'))
    total = quantity * price
    
    return (is_cancelled , total)

In [14]:
cancelled_total_rdd = retail_RDD_without_title.map(lambda x : cancelled_price(x))

In [15]:
cancelled_total_rdd.take(5)

[(False, 15.299999999999999),
 (False, 20.34),
 (False, 22.0),
 (False, 20.34),
 (False, 20.34)]

In [16]:
cancelled_total_rdd = cancelled_total_rdd.reduceByKey(lambda x,y : x+y)

In [17]:
cancelled_total_rdd.take(5)

[(False, 10644560.424000263), (True, -896812.4900000116)]